# Overview
### score

### csv_file


<hr>
### Action

- ランダムフォレストを使用
- 画像サイズはリサイズ
    600x600


# Module

In [1]:
import sys,os
import numpy as np
import pandas as pd
import json
#torch
import torch
import torch.nn as nn
# scripts
sys.path.append('../')
from scripts.utility_func import *
from scripts.Eval_Dice_coefficient import *
# models
from data.models import model_001
# experiments
from comet_ml import Experiment

In [3]:
NB = '003'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

# Path

In [16]:
DATASET_PATH = "../data_ignore/datasets/"
FEATURES_PATH = "../data/features/"
TRAIN_PATH  =  os.path.join(DATASET_PATH,"train")
TEST_PATH = os.path.join(DATASET_PATH,"test")
df_train = pd.read_csv(os.path.join(DATASET_PATH,"train.csv"))
df_sub = pd.read_csv(os.path.join(DATASET_PATH,'sample_submission.csv'))
df_info = pd.read_csv(os.path.join(FEATURES_PATH,'001-info_features.csv'))

# Experiment

In [13]:
json_data = json.load(open('../config/Comet.json','r'))
experiment = Experiment(
    api_key = json_data["comet"]["api_key"],
    project_name = json_data["comet"]["project_name"],
    workspace = json_data["comet"]["workspace"],
)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yusuketakahashi2001/kaggle-hubmap-hacking-the-kidney/573cd5e2eebe4cfe8185cec11c22c5cc



,image_file,width_pixels,height_pixels,anatomical_structures_segmention_file,glomerulus_segmentation_file,patient_number,race,ethnicity,sex,age,weight_kilograms,height_centimeters,bmi_kg/m^2,laterality,percent_cortex,percent_medulla
0,afa5e8098.tiff,43780,36800,afa5e8098-anatomical-structure.json,afa5e8098.json,67347,White,Not Hispanic or Latino,Female,58,59.0,160.0,23.0,Right,55,45
1,54f2eec69.tiff,22240,30440,54f2eec69-anatomical-structure.json,54f2eec69.json,67548,Black or African American,Not Hispanic or Latino,Male,58,79.9,190.5,22.0,Right,75,25
2,1e2425f28.tiff,32220,26780,1e2425f28-anatomical-structure.json,1e2425f28.json,63921,White,Not Hispanic or Latino,Male,48,131.5,193.0,35.3,Right,65,35
3,e79de561c.tiff,27020,16180,e79de561c-anatomical-structure.json,e79de561c.json,67026,Black or African American,Not Hispanic or Latino,Male,53,73.0,166.0,26.5,Left,55,45
4,c68fe75ea.tiff,19780,26840,c68fe75ea-anatomical-structure.json,c68fe75ea.json,67112,White,Not Hispanic or Latino,Male,56,91.2,167.6,32.5,Left,80,20
5,095bf7a1f.tiff,39000,38160,095bf7a1f-anatomical-structure.json,095bf7a1f.json,68250,White,Not Hispanic or Latino,Female,44,71.7,160.0,28.0,Right,65,35
6,26dc41664.tiff,42360,38160,26dc41664-anatomical-structure.json,26dc41664.json,68304,White,Not Hispanic or Latino,Female,66,71.3,167.6,25.4,Left,55,45
7,aaa6a05cc.tiff,13013,18484,aaa6a05cc-anatomical-structure.json,aaa6a05cc.json,65631,White,Not Hispanic or Latino,Female,73,NaN,NaN,NaN,Left,75,25
8,b9a3865fc.tiff,40429,31295,b9a3865fc-anatomical-structure.json,b9a3865fc.json,67347,White,Not Hispanic or Latino,Female,58,59.0,160.0,23.0,Right,55,45
9,cb2d976f4.tiff,49548,34940,cb2d976f4-anatomical-structure.json,cb2d976f4.json,67548,Black or African American,Not Hispanic or Latino,Male,58,79.9,190.5,22.0,Right,75,25
